In [1]:
import nltk
from nltk.corpus import webtext
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import math as m
ids = nltk.corpus.webtext.fileids()
ps = PorterStemmer()
from nltk.corpus import wordnet

In [2]:
def indexation(Pg):
  word = Pg.split(" ")
  word_filtered = [w for w in word if not w in stopwords.words('english')]
  word_Lem = [ps.stem(w) for w in word_filtered]
  word_Lem = [w.lower() for w in word_Lem]
  word_filtered = [w.lower() for w in word_filtered]
  return word_Lem , word_filtered

In [3]:
def poids_tf_idf(mot,pg):
  word_Lem , word_filtered = indexation(pg)
  mytexts = nltk.TextCollection([word_filtered, word_Lem])
  tf = mytexts.tf(mot,word_filtered)
  idf = mytexts.idf(mot)
  if idf == 0:
    idf=0.1
  tf_idf = mytexts.tf_idf(mot,word_filtered)
  return tf , idf , tf_idf 

In [4]:
def definition(mot):
  syns = wordnet.synsets(mot)
  if (len(syns) == 0):
    dfn =""
  else:
    dfn = syns[0].definition()
  return dfn

In [5]:
def recherche(mot):
  documents=[]
  poids=[]
  tfidf=[]
  freq=[]
  N = len(ids)
  for i in ids:
    with open(nltk.corpus.webtext.abspath(i),'r', encoding='utf-8', errors='ignore') as f:
      pg = f.read()
      if mot in pg:
        word_Lem , word_filtered = indexation(pg)
        if mot not in word_filtered:
          continue
        documents.append(i)
        tf , idf , tf_idf = poids_tf_idf(mot,pg)
        calc = (1+m.log10(tf))*(m.log10(N /idf))
        poids.append(calc)
        tfidf.append(tf_idf)
        freq.append(pg.count(mot))
  mi = -1e9
  j=-1
  for i in poids:
    if mi<i:
      mi = i
      j = poids.index(i)
          
          
  if (j == -1):
    pert=""
  else:
    pert = documents[j]

          
          
  
  return documents , poids , tfidf , freq , pert


In [6]:
def synant(mot):
  synonyms = []
  antonyms = []
  syns = wordnet.synsets(mot)
  for syn in syns:
    for l in syn.lemmas():
      synonyms.append(l.name())
      if l.antonyms():
        antonyms.append(l.antonyms()[0].name())
  return synonyms, antonyms

In [7]:
def fonction_grammaticale(mot):
  tokens = nltk.word_tokenize(mot)
  tags = nltk.pos_tag(tokens)
  fg = [tag[1] for tag in tags]
  return fg